In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../../loaders/imports.py
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

# Add the uoicorr directory to the path
sys.path.append('../../../uoicorr_run')

# Add the root directory of this repository
sys.path.append('../..')

from utils import gen_covariance, gen_beta2, gen_data, get_cov_list
from utils import selection_accuracy
from sklearn.linear_model import LassoLars, lasso_path, LinearRegression, LassoCV

from pyuoi.linear_model import UoI_Lasso
from pyc_based.lm import PycassoLasso

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
# %load ../../loaders/datgen.py
n_features = 50
n_samples = 150

sigma = gen_covariance(n_features, 0, n_features, 1, 0)
beta = gen_beta2(n_features, n_features, sparsity = 0.2, betawidth = np.inf)
X, X_test, y, y_test, ss = gen_data(n_samples, n_features, kappa = 100, 
									covariance = sigma, beta = beta)
X = StandardScaler().fit_transform(X)
y -= np.mean(y)

In [5]:
from sklearn.linear_model.coordinate_descent import _alpha_grid

In [6]:
lasso = PycassoLasso(fit_intercept=False, alphas = _alpha_grid(X, y))

In [7]:
lasso.fit(X, y)

In [8]:
from info_criteria import GIC, empirical_bayes

In [9]:
y_pred = lasso.coef_ @ X.T

In [10]:
# BIC scores
BIC_scores = np.array([GIC(y.ravel(), y_pred[i, :], np.count_nonzero(lasso.coef_[i, :]), np.log(n_samples))
                       for i in range(100)])

In [11]:
bfull = LinearRegression().fit(X, y).coef_.ravel()
ssq_hat = (y.T @ y - bfull.T @ X.T @ X @ bfull)/(X.shape[0] - X.shape[1])

In [14]:
EB_scores = np.array([empirical_bayes(X, y, y_pred[i, :], ssq_hat, lasso.coef_[i, :]) for i in range(100)])

In [16]:
EB_scores.shape

(100, 3)

In [18]:
np.argmin(EB_scores[:, 0])

0

In [27]:
_alpha_grid(X, y)

array([8.31701376, 7.75647487, 7.23371442, 6.74618628, 6.2915159 ,
       5.86748879, 5.47203968, 5.10324252, 4.75930105, 4.43854009,
       4.13939735, 3.86041582, 3.60023671, 3.35759281, 3.13130229,
       2.920263  , 2.72344704, 2.53989583, 2.36871535, 2.20907186,
       2.06018781, 1.92133805, 1.7918463 , 1.67108185, 1.55845653,
       1.45342177, 1.35546601, 1.26411214, 1.17891521, 1.09946027,
       1.02536033, 0.95625447, 0.89180612, 0.83170138, 0.77564749,
       0.72337144, 0.67461863, 0.62915159, 0.58674888, 0.54720397,
       0.51032425, 0.47593011, 0.44385401, 0.41393973, 0.38604158,
       0.36002367, 0.33575928, 0.31313023, 0.2920263 , 0.2723447 ,
       0.25398958, 0.23687153, 0.22090719, 0.20601878, 0.1921338 ,
       0.17918463, 0.16710819, 0.15584565, 0.14534218, 0.1355466 ,
       0.12641121, 0.11789152, 0.10994603, 0.10253603, 0.09562545,
       0.08918061, 0.08317014, 0.07756475, 0.07233714, 0.06746186,
       0.06291516, 0.05867489, 0.0547204 , 0.05103243, 0.04759

In [32]:
np.count_nonzero(lasso.coef_, axis = 1)[52]

10

In [21]:
np.argmax(BIC_scores)

1

In [30]:
EB_scores.shape

(100, 3)

In [31]:
np.argmin(EB_scores[1:, 0])

52